# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
import json

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

For this experiment I will be using Azure AutoML to make predictions regarding the likelyhood of a death event based on a patient's features as found in the Heart Failure Prediction data set on Kaggle. https://www.kaggle.com/andrewmvd/heart-failure-clinical-data

Features and target (DEATH_EVENT) of the dataset are:

* "age": Age of patient
* "anaemia": Decrease of red blood cells or hemoglobin (boolean)
* "creatinine_phosphokinase": Level of the CPK enzyme in the blood (mcg/L)
* "diabetes": If the patient has diabetes (boolean)
* "ejection_fraction": Percentage of blood leaving the heart at each contraction (percentage)
* "high_blood_pressure": If the patient has hypertension (boolean)
* "platelets": Platelets in the blood (kiloplatelets/mL)
* "serum_creatinine": Level of serum creatinine in the blood (mg/dL)
* "serum_sodium": Level of serum sodium in the blood (mEq/L)
* "sex": Woman or man (binary)
* "smoking": If the patient smokes or not (boolean)
* "time": Follow-up period (days)
* "DEATH_EVENT": If the patient deceased during the follow-up period (boolean)

In [2]:
# Load the workspace details from the config file.
ws = Workspace.from_config()

# Choose a name for experiment and create a new experiment.
experiment_name = 'udacity-exp-automl'
experiment=Experiment(ws, experiment_name)

In [3]:
# Load the registered dataset from the workspace.
dataset = Dataset.get_by_name(ws, name='heart_failure_data')

# Display the head of the dataset.
dataset.to_pandas_dataframe().head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [14]:
# Create compute cluster. If it does not exist then create one.
cpu_cluster_name = "udacity-comp-cst"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster")
    
except ComputeTargetException:    
    compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_DS12_V2", 
                                                           max_nodes=4,
                                                          idle_seconds_before_scaledown=600,
                                                          vm_priority='lowpriority')
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and configuration you used below.
* "n_cross_validations": I used 3 so that 3 different cross validation trainings were performed, each using 2/3 of the available data. For my small dataset and demostration purposes this value was ok. For larger datasets I would go go a higher number so that we get more data used in training and a smaller validation set.
* "primary_metric": For this experiment accuracy is a suitable metric for classification task.
* "enable_early_stopping": This will end the experiment is primary_metric is not improving, which will save me time waiting for experiment to end.  
* "experiment_timeout_hours": If expeirment takes more than 1 hour it will timeout. This is to avoid length experiments running unnecessarily.
* "max_concurrent_iterations": This is the max number of concurrent iterations of the expeirment. I set it to 4 as 1 iteration can run on 1 node of the compute cluster, which has 4 nodes.
* "max_cores_per_iteration": The max number of cpu threads to use for an iteration. I set this to -1 so as to use all available threads.
* "iteration_timeout_minutes": This is the timeout for each iteration. I set this to 15 minutes as I was not interested in anything take longer than this to run for my current experiment.
* "verbosity": I choose default the level of logging for my experiment. 
* "iterations": I set the maximum number of iterations for AutoML to run to be 40. This seemed like a reasonable number of iterations for the course without taking up too much time and compute on the Azure platform. 
* "task": The task is to predict a death event which is a classification task (yes/no).
* "compute_target": I set this to my computer cluster that I created earlier in the notebook.
* "training_data": This was set to the dataset that I loaded earlier.
* "label_column_name": This was set to the column DEATH_EVENT in the imported dataset. It is the column we wish to predict.

In [15]:
# TODO: Put your automl settings here
automl_settings = {
        "n_cross_validations": 3,
        "primary_metric": 'accuracy',
        "enable_early_stopping": True,
        "experiment_timeout_hours": 1.0,
        "max_concurrent_iterations": 4,
        "max_cores_per_iteration": -1,
        "iteration_timeout_minutes": 15,
        "verbosity": logging.INFO,
        "iterations": 40
        }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                                task = 'classification',
                                compute_target = cpu_cluster_name,
                                training_data = dataset,
                                label_column_name = 'DEATH_EVENT',
                                **automl_settings
                            )

In [16]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on udacity-comp-cst with default configuration
Running on remote compute: udacity-comp-cst
Parent Run ID: AutoML_9c66d14c-e778-4e7e-ad2d-8a425cd3b51a

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [17]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [19]:
# Retrieve your best automl model.
best_run, model = remote_run.get_output()

# Printt the status of the best run and the model.
print(best_run)
print(model)

Run(Experiment: udacity-exp-automl,
Id: AutoML_9c66d14c-e778-4e7e-ad2d-8a425cd3b51a_36,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_weight_fraction_leaf=0.0,
                                                                                                  n_estimators=600,
                

In [20]:
# Output the model steps.
model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('4',
                                             Pipeline(memory=None,
                                                      steps=[('minmaxscaler',
                                                              MinMaxScaler(copy=True,
                                                                           feature_range=(0,
                                                                                          1))),
                                                          

In [21]:
# More details regarding the model can be found below.
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['4', '6', '5', '31', '18', '8', '21', '26', '29'],
 'weights': [0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.2727272727272727,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091]}

4 - minmaxscaler
{'copy': True, 'feature_range': (0, 1)}

4 - randomforestclassifier
{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'log2',
 'max_leaf_nodes': None,
 'max_samples': None,


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [4]:
# Register the best model.
model = best_run.register_model(model_name='automl-best',model_path='outputs/model.pkl')

# Find an existing registered model.
# model = Model(ws, name='automl-best')

In [87]:
#Get an environment with AutoML packages installed.
environment = Environment.get(workspace=ws, name="AzureML-AutoML")

# Install scikit-learn to the environment.
for pip_package in ["scikit-learn"]:
    environment.python.conda_dependencies.add_pip_package(pip_package)

# Set the name of the endpoint service you will deploy. 
service_name = 'automl-best-sv'

# Define the inference and deployment config.
inference_config = InferenceConfig(entry_script='score.py', environment=environment)
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

# Deploy the model on Azure.
service = Model.deploy(workspace = ws,
                       name = service_name,
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = deployment_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.....................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [68]:
# RUN THIS CELL IF YOU NEED TO WORK WITH A PREVIOUSLY DEPLOYED ENDPOINT.
'''
from azureml.core import Webservice

# Print a list of deployed web services.
services = Webservice.list(ws)
print(services)

# Create a new instance of the deployed web service (if previously deployed)
service = Webservice(workspace=ws, name='automl-best-sv')
'''

[AciWebservice(workspace=Workspace.create(name='udacity-ws', subscription_id='12d8a290-6d86-4923-bc31-637a3ee790ed', resource_group='udacity-rg'), name=automl-2nd-best, image_id=None, compute_type=None, state=ACI, scoring_uri=None, tags=http://a4fcf479-b8a6-4646-b5f2-f5c45351c624.eastus.azurecontainer.io/score, properties={}, created_by={'runId': 'AutoML_9c66d14c-e778-4e7e-ad2d-8a425cd3b51a_4', 'hasInferenceSchema': 'True', 'hasHttps': 'False'}), AciWebservice(workspace=Workspace.create(name='udacity-ws', subscription_id='12d8a290-6d86-4923-bc31-637a3ee790ed', resource_group='udacity-rg'), name=automl-best-sv, image_id=None, compute_type=None, state=ACI, scoring_uri=None, tags=http://0891ed9d-f463-433b-86b9-45fc38bd6c16.eastus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'False', 'hasHttps': 'False'})]


TODO: In the cell below, send a request to the web service you deployed to test it.

In [88]:
# Enable application insights for the logging
service.update(enable_app_insights=True)

In [89]:
test_data = [{"age": 75, 
 "anaemia": 0, 
 "creatinine_phosphokinase": 582, 
 "diabetes": 0, 
 "ejection_fraction": 20, 
 "high_blood_pressure": 1, 
 "platelets": 265000, 
 "serum_creatinine": 1.9, 
 "serum_sodium": 130, 
 "sex": 1, 
 "smoking": 0, 
 "time": 4},

{"age": 75, 
 "anaemia": 0, 
 "creatinine_phosphokinase": 582, 
 "diabetes": 0, 
 "ejection_fraction": 20, 
 "high_blood_pressure": 1, 
 "platelets": 265000, 
 "serum_creatinine": 1.9, 
 "serum_sodium": 130, 
 "sex": 1, 
 "smoking": 0, 
 "time": 4}]

print(test_data)

[{'age': 75, 'anaemia': 0, 'creatinine_phosphokinase': 582, 'diabetes': 0, 'ejection_fraction': 20, 'high_blood_pressure': 1, 'platelets': 265000, 'serum_creatinine': 1.9, 'serum_sodium': 130, 'sex': 1, 'smoking': 0, 'time': 4}, {'age': 75, 'anaemia': 0, 'creatinine_phosphokinase': 582, 'diabetes': 0, 'ejection_fraction': 20, 'high_blood_pressure': 1, 'platelets': 265000, 'serum_creatinine': 1.9, 'serum_sodium': 130, 'sex': 1, 'smoking': 0, 'time': 4}]


In [90]:
# Create a JSON payload and then pass it to the new endpoint for inference.
input_payload = json.dumps({
    'data': test_data
})

# print(input_payload)

output = service.run(input_payload)
print(output)

[1, 1]


TODO: In the cell below, print the logs of the web service and delete the service

In [91]:
# Return the service logs for the new endpoint.
logs = service.get_logs()
print(logs)

2021-01-10T08:25:00,599695300+00:00 - gunicorn/run 
2021-01-10T08:25:00,622875000+00:00 - iot-server/run 
2021-01-10T08:25:00,658445500+00:00 - rsyslog/run 
2021-01-10T08:25:00,686475500+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [ ]:
# Delete the service.
service.delete()